In [1]:
import pandas as pd
import numpy as np
import os
import glob
import joblib
from tqdm import tqdm
import json
import subprocess
pd.set_option('display.max_rows', 200)

In [2]:
import model_predict as mp
self = mp.ModelPredict()

총 2 개의 모델이 있습니다.
['models/RF_84.pkl', 'models/RF_75.pkl']


In [3]:
self.label_ls

['human', 'human_voice', 'life', 'nature', 'song']

### mp4에서 wav를 추출
- wav파일 재저장
- 3초씩 1초마다 밀면서 잘라서 재저장

In [4]:
mp4_ls = [file for file in glob.glob(f'{self.video_path}*') if file.endswith(".mp4")]
mp4_name_ls = [os.path.basename(p).split('.')[0] for p in mp4_ls]
wav_name_ls = [os.path.basename(f).split('.')[0] for f in glob.glob(f'{self.wav_from_video_path}*') if f.endswith(".wav")]

# if mp4_name_ls != wav_name_ls:
#     for mp4_name in mp4_ls:
#         filename = os.path.basename(mp4_name).split('.')[0]
#         command = f"ffmpeg -i {mp4_name} -f wav -acodec libmp3lame -ar 44100 -vn {self.wav_from_video_path}{filename}.wav -y"
#         subprocess.call(command, shell=True)
# else:
#     pass

In [5]:
mp4_name_ls, wav_name_ls

(['test_video1', 'test_video3', 'test_video2'],
 ['test_video3', 'test_video1', 'test_video2'])

In [7]:
for ids in tqdm(range(len(mp4_name_ls))):
    self.cut_song_by_sec(f"{self.wav_from_video_path}{mp4_name_ls[ids]}.wav")

100%|██████████| 3/3 [00:30<00:00, 10.14s/it]


In [6]:
self.make_info_df()

,path,name,start,end
0,../sound_files/wav_cut3s/test_video1/0.0_3.0.wav,test_video1,0.0,3.0
1,../sound_files/wav_cut3s/test_video1/0.1_3.1.wav,test_video1,0.1,3.1
2,../sound_files/wav_cut3s/test_video1/0.2_3.2.wav,test_video1,0.2,3.2
3,../sound_files/wav_cut3s/test_video1/0.3_3.3.wav,test_video1,0.3,3.3
4,../sound_files/wav_cut3s/test_video1/0.4_3.4.wav,test_video1,0.4,3.4
...,...,...,...,...
32005,../sound_files/wav_cut3s/test_video3/1832.5_18...,test_video3,1832.5,1835.5
32006,../sound_files/wav_cut3s/test_video3/1832.6_18...,test_video3,1832.6,1835.6
32007,../sound_files/wav_cut3s/test_video3/1832.7_18...,test_video3,1832.7,1835.7
32008,../sound_files/wav_cut3s/test_video3/1832.8_18...,test_video3,1832.8,1835.8


### freq 데이터셋 만들기

In [7]:
freq_df = self.edit_frequency_df(filename="test_video2")
freq_df

100%|██████████| 6090/6090 [1:01:05<00:00,  1.66it/s]


,name,path,duration,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,...,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,start,end
0,test_video2,../sound_files/wav_cut3s/test_video2/0.0_3.0.wav,3.000000,0.359879,0.100569,0.124957,0.007491,1720.112860,1.131973e+06,1964.050485,...,-11.881530,80.763741,-2.900170,53.609623,-6.137068,71.502441,-3.232407,58.059181,0.0,3.0
1,test_video2,../sound_files/wav_cut3s/test_video2/0.1_3.1.wav,3.000000,0.366081,0.099609,0.128360,0.007012,1683.240838,1.037667e+06,1970.420985,...,-12.295943,74.697701,-2.935101,54.161629,-5.986935,73.779930,-3.258213,60.070370,0.1,3.1
2,test_video2,../sound_files/wav_cut3s/test_video2/0.2_3.2.wav,3.000000,0.365508,0.100959,0.130159,0.006687,1731.734366,1.246334e+06,1995.529608,...,-12.403485,74.488327,-2.659232,55.458858,-5.910663,75.080322,-3.147736,59.854229,0.2,3.2
3,test_video2,../sound_files/wav_cut3s/test_video2/0.3_3.3.wav,3.000000,0.374165,0.102498,0.132352,0.006239,1712.074452,1.253598e+06,2008.428591,...,-12.815776,75.125587,-2.312804,57.634125,-5.801310,77.152870,-2.755883,58.492641,0.3,3.3
4,test_video2,../sound_files/wav_cut3s/test_video2/0.4_3.4.wav,3.000000,0.369502,0.103374,0.134141,0.005884,1654.445861,1.245623e+06,2001.205203,...,-13.164048,70.199272,-2.409057,58.031860,-5.743892,77.995544,-2.610366,58.330498,0.4,3.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6085,test_video2,../sound_files/wav_cut3s/test_video2/608.5_611...,2.896009,0.491407,0.102523,0.021061,0.000098,642.938671,3.508309e+05,1253.971401,...,-2.288562,44.526703,5.609787,28.690744,1.857956,43.490543,3.834590,50.056442,608.5,611.5
6086,test_video2,../sound_files/wav_cut3s/test_video2/608.6_611...,2.796009,0.485045,0.101488,0.020224,0.000089,652.150643,3.554453e+05,1273.744978,...,-2.067557,43.786232,5.477502,28.868052,1.790399,42.910770,3.758516,48.592510,608.6,611.6
6087,test_video2,../sound_files/wav_cut3s/test_video2/608.7_611...,2.696009,0.491083,0.103067,0.020319,0.000093,658.616777,3.609396e+05,1285.096489,...,-1.937535,41.229454,5.461341,29.380672,1.421032,38.795601,3.238313,40.288136,608.7,611.7
6088,test_video2,../sound_files/wav_cut3s/test_video2/608.8_611...,2.596009,0.498000,0.101743,0.020364,0.000095,660.551493,3.716145e+05,1271.885556,...,-1.620676,39.395927,5.621856,29.816145,1.164235,36.103764,2.966022,37.855366,608.8,611.8


In [7]:
self.model_predict(self.model_ls[0], freq_df.iloc[[10]])

이 소리는 100.0의 확률로 'human_voice' 로 추정됩니다.


(array([0., 0., 1., 0.]), 'human_voice')

In [8]:
freq_df.to_csv(f"{self.file_path}freq_test_video2.csv", encoding='utf-8-sig')

-------------

# 영상 테스트

## 1번째 영상: 맛집 탐방 브이로그
- 주변 잡음 및 노래 소리가 심함

In [11]:
result_df, pct_df = self.predict_check_result(filename='test_video1', start=7*60, end=8*60)

  0%|          | 0/60 [00:00<?, ?it/s]

['human', 'human_voice', 'life', 'nature', 'song']
총 2 개의 모델이 있습니다.
['models/RF_75.pkl', 'models/RF_84.pkl']


  3%|▎         | 2/60 [00:00<00:04, 13.20it/s]

이 소리는 70.0의 확률로 'song' 로 추정됩니다.
이 소리는 40.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 70.0의 확률로 'song' 로 추정됩니다.
이 소리는 90.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 90.0의 확률로 'song' 로 추정됩니다.
이 소리는 80.0의 확률로 'song' 로 추정됩니다.
----------------


 10%|█         | 6/60 [00:00<00:04, 13.33it/s]

...

이 소리는 50.0의 확률로 'song' 로 추정됩니다.
이 소리는 60.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 80.0의 확률로 'song' 로 추정됩니다.
이 소리는 80.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 60.0의 확률로 'song' 로 추정됩니다.
이 소리는 80.0의 확률로 'song' 로 추정됩니다.
----------------


 97%|█████████▋| 58/60 [00:04<00:00, 13.34it/s]

이 소리는 80.0의 확률로 'song' 로 추정됩니다.
이 소리는 90.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 90.0의 확률로 'song' 로 추정됩니다.
이 소리는 60.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 80.0의 확률로 'song' 로 추정됩니다.
이 소리는 70.0의 확률로 'song' 로 추정됩니다.
----------------


100%|██████████| 60/60 [00:04<00:00, 13.27it/s]


이 소리는 60.0의 확률로 'song' 로 추정됩니다.
이 소리는 70.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 60.0의 확률로 'song' 로 추정됩니다.
이 소리는 50.0의 확률로 'song' 로 추정됩니다.
----------------
'('test_video1', 420)' 은 55.00000000000001% 의 확률로 song로 추정됩니다.
'('test_video1', 421)' 은 80.0% 의 확률로 song로 추정됩니다.
'('test_video1', 422)' 은 85.0% 의 확률로 song로 추정됩니다.
'('test_video1', 423)' 은 45.0% 의 확률로 human_voice로 추정됩니다.
'('test_video1', 424)' 은 45.0% 의 확률로 human_voice로 추정됩니다.
'('test_video1', 425)' 은 55.00000000000001% 의 확률로 song로 추정됩니다.
'('test_video1', 426)' 은 80.0% 의 확률로 song로 추정됩니다.
'('test_video1', 427)' 은 60.0% 의 확률로 song로 추정됩니다.
'('test_video1', 428)' 은 60.0% 의 확률로 song로 추정됩니다.
'('test_video1', 429)' 은 55.00000000000001% 의 확률로 song로 추정됩니다.
'('test_video1', 430)' 은 60.0% 의 확률로 human_voice로 추정됩니다.
'('test_video1', 431)' 은 45.0% 의 확률로 human_voice로 추정됩니다.
'('test_video1', 432)' 은 55.00000000000001% 의 확률로 song로 추정됩니다.
'('test_video1', 433)' 은 45.0% 의 확률로 song로 추정됩니다.
'('test_video1', 434)' 은 45.0% 의 확률로 human_voice로 추정됩니다.
'

In [12]:
pct_df

Human  Non-human
name        start                  
test_video1 420     15.0       85.0
            421     20.0       80.0
            422     15.0       85.0
            423     70.0       30.0
            424     70.0       30.0
            425     40.0       60.0
            426     20.0       80.0
            427     35.0       65.0
            428     40.0       60.0
            429     45.0       55.0
            430     65.0       35.0
            431     75.0       25.0
            432     45.0       55.0
            433     55.0       45.0
            434     55.0       45.0
            435     70.0       30.0
            436     40.0       60.0
            437     45.0       55.0
            438     80.0       20.0
            439     60.0       40.0
            440     75.0       25.0
            441     70.0       30.0
            442     65.0       35.0
            443     70.0       30.0
            444     40.0       60.0
            445      0.0      100.0
            446     10.0       90.0
            447     20.0       80.0
            448     25.0       75.0
            449     20.0       80.0
            450     20.0       80.0
            451     20.0       80.0
            452     40.0       60.0
            453     25.0       75.0
            454     45.0       55.0
            455     45.0       55.0
            456     30.0       70.0
            457     10.0       90.0
            458     15.0       85.0
            459     35.0       65.0
            460     45.0       55.0
            461     25.0       75.0
            462     35.0       65.0
            463     40.0       60.0
            464     25.0       75.0
            465     45.0       55.0
            466     30.0       70.0
            467     30.0       70.0
            468     25.0       75.0
            469     50.0       50.0
            470     20.0       80.0
            471     20.0       80.0
            472     30.0       70.0
            473     15.0       85.0
            474     10.0       90.0
            475      5.0       95.0
            476     20.0       80.0
            477     15.0       85.0
            478     25.0       75.0
            479     30.0       70.0

In [15]:
result_df.groupby(['model','start','y_pred']).mean()[:60]

human  human_voice  life  nature  song
model start y_pred                                             
RF_75 420   song           0.1          0.0   0.2     0.0   0.7
      421   song           0.3          0.0   0.0     0.0   0.7
      422   song           0.0          0.1   0.0     0.0   0.9
      423   human_voice    0.0          0.7   0.0     0.0   0.3
      424   human_voice    0.1          0.7   0.0     0.0   0.2
      425   song           0.0          0.4   0.1     0.0   0.5
      426   song           0.0          0.3   0.0     0.0   0.7
      427   human_voice    0.0          0.5   0.0     0.0   0.5
      428   song           0.1          0.1   0.0     0.0   0.8
      429   song           0.0          0.3   0.0     0.0   0.7
      430   human_voice    0.0          0.8   0.0     0.0   0.2
      431   human_voice    0.1          0.8   0.0     0.0   0.1
      432   human_voice    0.0          0.5   0.0     0.0   0.5
      433   human_voice    0.1          0.6   0.0     0.0   0.3
      434   human_voice    0.0          0.8   0.0     0.0   0.2
      435   human_voice    0.0          0.8   0.0     0.0   0.2
      436   song           0.2          0.3   0.1     0.0   0.4
      437   human_voice    0.1          0.5   0.0     0.0   0.4
      438   human_voice    0.0          0.9   0.0     0.0   0.1
      439   human_voice    0.1          0.6   0.1     0.0   0.2
      440   human_voice    0.1          0.7   0.0     0.0   0.2
      441   human_voice    0.1          0.7   0.0     0.0   0.2
      442   human_voice    0.1          0.7   0.1     0.0   0.1
      443   human_voice    0.0          0.8   0.2     0.0   0.0
      444   human_voice    0.0          0.5   0.1     0.0   0.4
      445   song           0.0          0.0   0.1     0.0   0.9
      446   song           0.0          0.2   0.0     0.0   0.8
      447   song           0.0          0.3   0.0     0.0   0.7
      448   song           0.1          0.1   0.1     0.0   0.7
      449   song           0.1          0.1   0.0     0.0   0.8
      450   song           0.1          0.1   0.1     0.0   0.7
      451   song           0.0          0.2   0.0     0.0   0.8
      452   song           0.1          0.4   0.0     0.0   0.5
      453   song           0.1          0.4   0.0     0.0   0.5
      454   human_voice    0.0          0.6   0.0     0.0   0.4
      455   song           0.1          0.3   0.0     0.0   0.6
      456   song           0.0          0.4   0.0     0.0   0.6
      457   song           0.0          0.2   0.1     0.0   0.7
      458   song           0.0          0.2   0.1     0.0   0.7
      459   human_voice    0.1          0.3   0.3     0.0   0.3
      460   human_voice    0.0          0.6   0.0     0.0   0.4
      461   human_voice    0.0          0.4   0.2     0.0   0.4
      462   human_voice    0.0          0.5   0.1     0.0   0.4
      463   human_voice    0.0          0.5   0.2     0.0   0.3
      464   song           0.0          0.4   0.0     0.0   0.6
      465   human_voice    0.0          0.6   0.0     0.0   0.4
      466   human_voice    0.0          0.5   0.1     0.0   0.4
      467   human_voice    0.1          0.4   0.1     0.0   0.4
      468   song           0.1          0.3   0.1     0.0   0.5
      469   human_voice    0.0          0.4   0.2     0.0   0.4
      470   life           0.1          0.1   0.5     0.0   0.3
      471   song           0.1          0.1   0.3     0.0   0.5
      472   song           0.1          0.2   0.2     0.0   0.5
      473   song           0.1          0.1   0.0     0.0   0.8
      474   song           0.1          0.1   0.2     0.0   0.6
      475   song           0.1          0.0   0.1     0.0   0.8
      476   song           0.1          0.0   0.0     0.0   0.9
      477   song           0.1          0.1   0.0     0.0   0.8
      478   song           0.1          0.2   0.1     0.0   0.6
      479   song           0.1          0.3   0.0     0.0   0.6

## 2번째 영상: 먹방 컨텐츠
- 목소리가 끊이지않고 거의 계속 나옴
- 배경소리 거의 없이 조용한 편

In [17]:
result_df2, pct_df2 = self.predict_check_result(filename='test_video2', start=100, end=200)

  0%|          | 0/100 [00:00<?, ?it/s]

['human', 'human_voice', 'life', 'nature', 'song']
총 2 개의 모델이 있습니다.
['models/RF_75.pkl', 'models/RF_84.pkl']


  2%|▏         | 2/100 [00:00<00:10,  9.37it/s]

이 소리는 50.0의 확률로 'song' 로 추정됩니다.
이 소리는 50.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 50.0의 확률로 'life' 로 추정됩니다.
이 소리는 70.0의 확률로 'life' 로 추정됩니다.
----------------


  4%|▍         | 4/100 [00:00<00:08, 11.48it/s]

...

이 소리는 80.0의 확률로 'life' 로 추정됩니다.
이 소리는 80.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 60.0의 확률로 'life' 로 추정됩니다.
이 소리는 80.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 60.0의 확률로 'life' 로 추정됩니다.
이 소리는 60.0의 확률로 'life' 로 추정됩니다.
----------------


 98%|█████████▊| 98/100 [00:07<00:00, 12.93it/s]

이 소리는 70.0의 확률로 'life' 로 추정됩니다.
이 소리는 80.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 60.0의 확률로 'life' 로 추정됩니다.
이 소리는 80.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 40.0의 확률로 'life' 로 추정됩니다.
이 소리는 50.0의 확률로 'human' 로 추정됩니다.
----------------


100%|██████████| 100/100 [00:07<00:00, 12.71it/s]


이 소리는 60.0의 확률로 'life' 로 추정됩니다.
이 소리는 30.0의 확률로 'life' 로 추정됩니다.
----------------
'('test_video2', 100)' 은 45.0% 의 확률로 life로 추정됩니다.
'('test_video2', 101)' 은 60.0% 의 확률로 life로 추정됩니다.
'('test_video2', 102)' 은 40.0% 의 확률로 life로 추정됩니다.
'('test_video2', 103)' 은 55.00000000000001% 의 확률로 life로 추정됩니다.
'('test_video2', 104)' 은 60.0% 의 확률로 song로 추정됩니다.
'('test_video2', 105)' 은 70.0% 의 확률로 song로 추정됩니다.
'('test_video2', 106)' 은 85.0% 의 확률로 song로 추정됩니다.
'('test_video2', 107)' 은 55.00000000000001% 의 확률로 song로 추정됩니다.
'('test_video2', 108)' 은 60.0% 의 확률로 song로 추정됩니다.
'('test_video2', 109)' 은 50.0% 의 확률로 song로 추정됩니다.
'('test_video2', 110)' 은 50.0% 의 확률로 life로 추정됩니다.
'('test_video2', 111)' 은 50.0% 의 확률로 life로 추정됩니다.
'('test_video2', 112)' 은 70.0% 의 확률로 song로 추정됩니다.
'('test_video2', 113)' 은 85.0% 의 확률로 song로 추정됩니다.
'('test_video2', 114)' 은 80.0% 의 확률로 song로 추정됩니다.
'('test_video2', 115)' 은 50.0% 의 확률로 song로 추정됩니다.
'('test_video2', 116)' 은 45.0% 의 확률로 song로 추정됩니다.
'('test_video2', 117)' 은 35.0% 의 확률로 human로

In [20]:
pct_df2

Human  Non-human
name        start                  
test_video2 100     10.0       90.0
            101     10.0       90.0
            102     10.0       90.0
            103      5.0       95.0
            104     20.0       80.0
            105     15.0       85.0
            106      0.0      100.0
            107     15.0       85.0
            108     10.0       90.0
            109     20.0       80.0
            110     15.0       85.0
            111      5.0       95.0
            112      5.0       95.0
            113     10.0       90.0
            114     20.0       80.0
            115     50.0       50.0
            116     50.0       50.0
            117     70.0       30.0
            118     80.0       20.0
            119     95.0        5.0
            120     55.0       45.0
            121     45.0       55.0
            122     15.0       85.0
            123     20.0       80.0
            124     10.0       90.0
            125     10.0       90.0
            126     10.0       90.0
            127      5.0       95.0
            128      5.0       95.0
            129     15.0       85.0
            130     10.0       90.0
            131      5.0       95.0
            132     10.0       90.0
            133     30.0       70.0
            134     25.0       75.0
            135     40.0       60.0
            136     80.0       20.0
            137     95.0        5.0
            138     85.0       15.0
            139     95.0        5.0
            140     80.0       20.0
            141     65.0       35.0
            142     25.0       75.0
            143     20.0       80.0
            144     20.0       80.0
            145     30.0       70.0
            146     40.0       60.0
            147     50.0       50.0
            148     35.0       65.0
            149     30.0       70.0
            150     30.0       70.0
            151     25.0       75.0
            152     45.0       55.0
            153     60.0       40.0
            154     35.0       65.0
            155     45.0       55.0
            156     40.0       60.0
            157     15.0       85.0
            158     20.0       80.0
            159     15.0       85.0
            160     35.0       65.0
            161     40.0       60.0
            162     15.0       85.0
            163     20.0       80.0
            164     45.0       55.0
            165     50.0       50.0
            166     40.0       60.0
            167     10.0       90.0
            168     15.0       85.0
            169      0.0      100.0
            170      5.0       95.0
            171     20.0       80.0
            172      0.0      100.0
            173      0.0      100.0
            174     10.0       90.0
            175     25.0       75.0
            176     10.0       90.0
            177      5.0       95.0
            178     10.0       90.0
            179     25.0       75.0
            180     15.0       85.0
            181     20.0       80.0
            182     10.0       90.0
            183     15.0       85.0
            184      5.0       95.0
            185      5.0       95.0
            186     10.0       90.0
            187      5.0       95.0
            188     25.0       75.0
            189     10.0       90.0
            190     10.0       90.0
            191      5.0       95.0
            192      5.0       95.0
            193      5.0       95.0
            194      5.0       95.0
            195      5.0       95.0
            196      0.0      100.0
            197     15.0       85.0
            198     40.0       60.0
            199     25.0       75.0

In [21]:
result_df2.groupby(['model','start','y_pred']).mean()[:100]

human  human_voice  life  nature  song
model start y_pred                                             
RF_75 100   song           0.1          0.0   0.4     0.0   0.5
      101   life           0.2          0.0   0.5     0.0   0.3
      102   life           0.1          0.0   0.5     0.1   0.3
      103   life           0.1          0.0   0.5     0.0   0.4
      104   song           0.2          0.2   0.0     0.0   0.6
      105   song           0.2          0.0   0.0     0.0   0.8
      106   song           0.0          0.0   0.1     0.0   0.9
      107   song           0.1          0.2   0.2     0.0   0.5
      108   life           0.1          0.1   0.4     0.0   0.4
      109   song           0.0          0.2   0.2     0.0   0.6
      110   song           0.1          0.1   0.3     0.0   0.5
      111   life           0.0          0.0   0.5     0.0   0.5
      112   song           0.0          0.0   0.2     0.0   0.8
      113   song           0.0          0.1   0.0     0.0   0.9
      114   song           0.0          0.3   0.0     0.0   0.7
      115   human_voice    0.0          0.7   0.0     0.0   0.3
      116   song           0.0          0.4   0.0     0.0   0.6
      117   human_voice    0.2          0.5   0.1     0.0   0.2
      118   human_voice    0.3          0.4   0.0     0.0   0.3
      119   human_voice    0.1          0.9   0.0     0.0   0.0
      120   human_voice    0.2          0.5   0.1     0.0   0.2
      121   human_voice    0.0          0.7   0.0     0.0   0.3
      122   song           0.0          0.3   0.0     0.0   0.7
      123   song           0.0          0.3   0.0     0.0   0.7
      124   song           0.0          0.2   0.0     0.0   0.8
      125   song           0.0          0.2   0.2     0.0   0.6
      126   song           0.0          0.1   0.2     0.0   0.7
      127   life           0.0          0.1   0.5     0.0   0.4
      128   song           0.0          0.0   0.3     0.0   0.7
      129   song           0.1          0.0   0.0     0.0   0.9
      130   song           0.0          0.1   0.0     0.0   0.9
      131   song           0.0          0.1   0.0     0.0   0.9
      132   song           0.0          0.2   0.0     0.0   0.8
      133   song           0.0          0.4   0.1     0.0   0.5
      134   song           0.3          0.1   0.0     0.0   0.6
      135   song           0.1          0.2   0.0     0.0   0.7
      136   human_voice    0.0          0.9   0.0     0.0   0.1
      137   human_voice    0.2          0.8   0.0     0.0   0.0
      138   human_voice    0.2          0.7   0.1     0.0   0.0
      139   human_voice    0.1          0.8   0.1     0.0   0.0
      140   human_voice    0.0          0.7   0.2     0.0   0.1
      141   human_voice    0.0          0.5   0.4     0.0   0.1
      142   life           0.2          0.2   0.5     0.0   0.1
      143   life           0.2          0.0   0.7     0.0   0.1
      144   life           0.2          0.1   0.4     0.0   0.3
      145   human          0.4          0.2   0.3     0.1   0.0
      146   life           0.2          0.1   0.5     0.1   0.1
      147   human          0.5          0.1   0.3     0.0   0.1
      148   life           0.2          0.2   0.3     0.0   0.3
      149   life           0.2          0.2   0.4     0.0   0.2
      150   life           0.0          0.2   0.6     0.0   0.2
      151   life           0.1          0.1   0.6     0.0   0.2
      152   life           0.0          0.1   0.8     0.0   0.1
      153   human          0.6          0.0   0.4     0.0   0.0
      154   life           0.4          0.1   0.5     0.0   0.0
      155   life           0.2          0.3   0.5     0.0   0.0
      156   life           0.2          0.2   0.5     0.0   0.1
      157   life           0.2          0.0   0.7     0.0   0.1
      158   life           0.3          0.0   0.7     0.0   0.0
      159   life           0.2          0.1   0.7     0.0   0.0
      160   song           0.2          0.1   0.3     0.0

## 3번째 영상: 영상편집 교육 컨텐츠
- 목소리가 끊이지않고 거의 계속 나옴
- 배경소리 거의 없이 조용한 편

In [22]:
result_df3, pct_df3 = self.predict_check_result(filename='test_video3', end=120)

  0%|          | 0/120 [00:00<?, ?it/s]

['human', 'human_voice', 'life', 'nature', 'song']
총 2 개의 모델이 있습니다.
['models/RF_75.pkl', 'models/RF_84.pkl']


  2%|▏         | 2/120 [00:00<00:10, 10.74it/s]

이 소리는 90.0의 확률로 'life' 로 추정됩니다.
이 소리는 90.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 80.0의 확률로 'life' 로 추정됩니다.
이 소리는 100.0의 확률로 'life' 로 추정됩니다.
----------------
이 소리는 70.0의 확률로 'life' 로 추정됩니다.


  3%|▎         | 4/120 [00:00<00:09, 11.83it/s]

...

이 소리는 50.0의 확률로 'human_voice' 로 추정됩니다.
이 소리는 50.0의 확률로 'human_voice' 로 추정됩니다.
----------------
이 소리는 40.0의 확률로 'human_voice' 로 추정됩니다.
이 소리는 60.0의 확률로 'human_voice' 로 추정됩니다.
----------------
이 소리는 50.0의 확률로 'song' 로 추정됩니다.
이 소리는 50.0의 확률로 'human_voice' 로 추정됩니다.
----------------


 98%|█████████▊| 118/120 [00:09<00:00, 12.99it/s]

이 소리는 50.0의 확률로 'song' 로 추정됩니다.
이 소리는 70.0의 확률로 'human_voice' 로 추정됩니다.
----------------
이 소리는 70.0의 확률로 'human_voice' 로 추정됩니다.
이 소리는 70.0의 확률로 'human_voice' 로 추정됩니다.
----------------
이 소리는 90.0의 확률로 'human_voice' 로 추정됩니다.
이 소리는 70.0의 확률로 'human_voice' 로 추정됩니다.
----------------


100%|██████████| 120/120 [00:09<00:00, 12.62it/s]


이 소리는 60.0의 확률로 'song' 로 추정됩니다.
이 소리는 60.0의 확률로 'song' 로 추정됩니다.
----------------
이 소리는 50.0의 확률로 'human_voice' 로 추정됩니다.
이 소리는 60.0의 확률로 'song' 로 추정됩니다.
----------------
'('test_video3', 0)' 은 90.0% 의 확률로 life로 추정됩니다.
'('test_video3', 1)' 은 90.0% 의 확률로 life로 추정됩니다.
'('test_video3', 2)' 은 85.0% 의 확률로 life로 추정됩니다.
'('test_video3', 3)' 은 75.0% 의 확률로 life로 추정됩니다.
'('test_video3', 4)' 은 70.0% 의 확률로 life로 추정됩니다.
'('test_video3', 5)' 은 55.00000000000001% 의 확률로 life로 추정됩니다.
'('test_video3', 6)' 은 35.0% 의 확률로 life로 추정됩니다.
'('test_video3', 7)' 은 50.0% 의 확률로 song로 추정됩니다.
'('test_video3', 8)' 은 60.0% 의 확률로 song로 추정됩니다.
'('test_video3', 9)' 은 50.0% 의 확률로 song로 추정됩니다.
'('test_video3', 10)' 은 75.0% 의 확률로 song로 추정됩니다.
'('test_video3', 11)' 은 45.0% 의 확률로 song로 추정됩니다.
'('test_video3', 12)' 은 45.0% 의 확률로 song로 추정됩니다.
'('test_video3', 13)' 은 35.0% 의 확률로 song로 추정됩니다.
'('test_video3', 14)' 은 40.0% 의 확률로 life로 추정됩니다.
'('test_video3', 15)' 은 35.0% 의 확률로 life로 추정됩니다.
'('test_video3', 16)' 은 60.0% 의 확률로 song로 추정

In [23]:
pct_df3

Human  Non-human
name        start                  
test_video3 0        5.0       95.0
            1        5.0       95.0
            2       15.0       85.0
            3        5.0       95.0
            4       10.0       90.0
            5       10.0       90.0
            6       35.0       65.0
            7       40.0       60.0
            8       25.0       75.0
            9       10.0       90.0
            10      20.0       80.0
            11      35.0       65.0
            12      35.0       65.0
            13      35.0       65.0
            14      25.0       75.0
            15      35.0       65.0
            16      10.0       90.0
            17      20.0       80.0
            18       5.0       95.0
            19      10.0       90.0
            20      15.0       85.0
            21      25.0       75.0
            22      25.0       75.0
            23      25.0       75.0
            24      25.0       75.0
            25      45.0       55.0
            26      20.0       80.0
            27      10.0       90.0
            28       0.0      100.0
            29      10.0       90.0
            30      10.0       90.0
            31       5.0       95.0
            32      10.0       90.0
            33      10.0       90.0
            34      20.0       80.0
            35      65.0       35.0
            36      85.0       15.0
            37      75.0       25.0
            38      85.0       15.0
            39      85.0       15.0
            40      80.0       20.0
            41      70.0       30.0
            42      65.0       35.0
            43      70.0       30.0
            44      75.0       25.0
            45      65.0       35.0
            46      45.0       55.0
            47      70.0       30.0
            48      35.0       65.0
            49      60.0       40.0
            50      80.0       20.0
            51      80.0       20.0
            52      75.0       25.0
            53      80.0       20.0
            54      60.0       40.0
            55      55.0       45.0
            56      40.0       60.0
            57      70.0       30.0
            58      65.0       35.0
            59      85.0       15.0
            60      70.0       30.0
            61      75.0       25.0
            62      80.0       20.0
            63      70.0       30.0
            64      80.0       20.0
            65      85.0       15.0
            66      70.0       30.0
            67      65.0       35.0
            68      65.0       35.0
            69      75.0       25.0
            70      65.0       35.0
            71      90.0       10.0
            72      85.0       15.0
            73      80.0       20.0
            74      60.0       40.0
            75      70.0       30.0
            76      70.0       30.0
            77      80.0       20.0
            78      65.0       35.0
            79      70.0       30.0
            80      40.0       60.0
            81      50.0       50.0
            82      65.0       35.0
            83      75.0       25.0
            84      95.0        5.0
            85      75.0       25.0
            86      70.0       30.0
            87      70.0       30.0
            88      50.0       50.0
            89      55.0       45.0
            90      60.0       40.0
            91      70.0       30.0
            92      80.0       20.0
            93      75.0       25.0
            94      60.0       40.0
            95      80.0       20.0
            96      55.0       45.0
            97      65.0       35.0
            98      60.0       40.0
            99      70.0       30.0
            100     40.0       60.0
            101     65.0       35.0
            102     45.0       55.0
            103     45.0       55.0
            104     45.0       55.0
            105     45.0       55.0
            106     60.0       40.0
            107     60.0       40.0
            108     50.0       50.0
            109     60.

In [24]:
result_df3.groupby(['model','start','y_pred']).mean()[:120]

human  human_voice  life  nature  song
model start y_pred                                             
RF_75 0     life           0.1          0.0   0.9     0.0   0.0
      1     life           0.1          0.0   0.8     0.0   0.1
      2     life           0.2          0.1   0.7     0.0   0.0
      3     life           0.0          0.1   0.7     0.0   0.2
      4     life           0.2          0.0   0.7     0.1   0.0
      5     life           0.0          0.1   0.5     0.2   0.2
      6     life           0.2          0.1   0.4     0.0   0.3
      7     song           0.1          0.3   0.1     0.0   0.5
      8     song           0.0          0.2   0.1     0.0   0.7
      9     song           0.0          0.1   0.1     0.0   0.8
      10    song           0.0          0.2   0.1     0.0   0.7
      11    life           0.2          0.2   0.3     0.0   0.3
      12    song           0.3          0.2   0.1     0.0   0.4
      13    human_voice    0.1          0.5   0.1     0.0   0.3
      14    life           0.1          0.2   0.4     0.0   0.3
      15    life           0.1          0.2   0.4     0.0   0.3
      16    song           0.0          0.1   0.0     0.2   0.7
      17    song           0.0          0.3   0.1     0.0   0.6
      18    song           0.0          0.0   0.2     0.0   0.8
      19    song           0.0          0.0   0.4     0.0   0.6
      20    song           0.0          0.1   0.2     0.0   0.7
      21    song           0.1          0.1   0.1     0.1   0.6
      22    song           0.2          0.1   0.1     0.0   0.6
      23    song           0.2          0.0   0.2     0.0   0.6
      24    song           0.2          0.2   0.2     0.0   0.4
      25    song           0.0          0.3   0.1     0.0   0.6
      26    song           0.2          0.1   0.1     0.0   0.6
      27    song           0.0          0.2   0.2     0.0   0.6
      28    song           0.0          0.0   0.2     0.1   0.7
      29    song           0.0          0.2   0.2     0.0   0.6
      30    song           0.0          0.2   0.2     0.2   0.4
      31    song           0.0          0.1   0.3     0.0   0.6
      32    song           0.1          0.1   0.1     0.0   0.7
      33    song           0.2          0.0   0.2     0.0   0.6
      34    song           0.1          0.2   0.3     0.0   0.4
      35    human_voice    0.0          1.0   0.0     0.0   0.0
      36    human_voice    0.0          1.0   0.0     0.0   0.0
      37    human_voice    0.0          0.9   0.0     0.0   0.1
      38    human_voice    0.0          0.9   0.0     0.0   0.1
      39    human_voice    0.0          0.9   0.1     0.0   0.0
      40    human_voice    0.0          0.9   0.0     0.0   0.1
      41    human_voice    0.0          0.8   0.0     0.0   0.2
      42    human_voice    0.0          0.5   0.1     0.0   0.4
      43    human_voice    0.1          0.8   0.0     0.0   0.1
      44    human_voice    0.0          0.7   0.0     0.0   0.3
      45    human_voice    0.1          0.5   0.0     0.0   0.4
      46    human_voice    0.1          0.4   0.1     0.0   0.4
      47    human_voice    0.0          0.6   0.0     0.0   0.4
      48    human_voice    0.0          0.5   0.0     0.0   0.5
      49    human_voice    0.0          0.6   0.1     0.0   0.3
      50    human_voice    0.1          0.7   0.1     0.0   0.1
      51    human_voice    0.1          0.7   0.0     0.0   0.2
      52    human_voice    0.0          0.9   0.0     0.0   0.1
      53    human_voice    0.0          0.9   0.0     0.0   0.1
      54    human_voice    0.1          0.5   0.0     0.0   0.4
      55    song           0.2          0.3   0.0     0.0   0.5
      56    song           0.0          0.4   0.0     0.0   0.6
      57    human_voice    0.0          0.7   0.0     0.0   0.3
      58    human_voice    0.0          0.7   0.0     0.0   0.3
      59    human_voice    0.0          0.9   0.0     0.0   0.1
      60    human_voice    0.0          0.9   0.0     0.0